In [26]:
import numpy as np
import tensorflow as tf
import os

In [27]:
import matplotlib.pyplot as plt
import _pickle
import time

In [28]:
# todo
# compare likelihoods
# test with less popular features
# use different priors
# multiclass logistic regression significance tests
# fit mixture of gaussian vs gaussian
# check if weight activity killed
# tansfer learning for new class and robustness

In [77]:
file = open('true_class_saliency_mask_trial2_10.pkl', "rb")
# file = open('mean_diff_saliency_mask_vanilla_temp1.pkl', "rb")
true_saliencies = _pickle.load(file) 
file = open('mean.pkl', "rb")
gaussian_means = _pickle.load(file) 
file = open('covariance.pkl', "rb")
gaussian_covariances = _pickle.load(file)

with open('turker_prob.pkl', "rb") as handle:
    turkers = _pickle.load(handle) 

with open('test_set_17_227.pkl', "rb") as handle:
    test_set = _pickle.load(handle) 
    
with open('top_2_features.pkl', "rb") as handle:
    turker_count= _pickle.load(handle) 

In [78]:
turker_count = {'Alec Baldwin': {'beard': 11,
  'cheek': 67,
  'ears': 19,
  'eye brows': 94,
  'eyes': 265,
  'hairline': 46,
  'lips': 164,
  'moustache': 11,
  'nose': 161},
 'America Ferrera': {'beard': 1,
  'cheek': 134,
  'ears': 9,
  'eye brows': 85,
  'eyes': 154,
  'hairline': 32,
  'lips': 260,
  'moustache': 7,
  'nose': 156},
 'Angie Harmon': {'beard': 8,
  'cheek': 123,
  'ears': 10,
  'eye brows': 125,
  'eyes': 206,
  'hairline': 27,
  'lips': 215,
  'moustache': 9,
  'nose': 115},
 'Anne Hathaway': {'beard': 4,
  'cheek': 40,
  'ears': 7,
  'eye brows': 90,
  'eyes': 256,
  'hairline': 42,
  'lips': 312,
  'moustache': 2,
  'nose': 85},
 'Bill Hader': {'beard': 9,
  'cheek': 62,
  'ears': 15,
  'eye brows': 126,
  'eyes': 270,
  'hairline': 71,
  'lips': 153,
  'moustache': 11,
  'nose': 121},
 'Cheryl Hines': {'beard': 6,
  'cheek': 77,
  'ears': 15,
  'eye brows': 46,
  'eyes': 224,
  'hairline': 48,
  'lips': 299,
  'moustache': 3,
  'nose': 120},
 'Daniel Radcliffe': {'beard': 14,
  'cheek': 59,
  'ears': 20,
  'eye brows': 178,
  'eyes': 258,
  'hairline': 61,
  'lips': 119,
  'moustache': 16,
  'nose': 113},
 'Fran Drescher': {'beard': 6,
  'cheek': 94,
  'ears': 10,
  'eye brows': 122,
  'eyes': 205,
  'hairline': 35,
  'lips': 268,
  'moustache': 11,
  'nose': 87},
 'Gerard Butler': {'beard': 167,
  'cheek': 42,
  'ears': 21,
  'eye brows': 57,
  'eyes': 271,
  'hairline': 26,
  'lips': 73,
  'moustache': 67,
  'nose': 114},
 'Jennifer Aniston': {'beard': 8,
  'cheek': 82,
  'ears': 17,
  'eye brows': 64,
  'eyes': 283,
  'hairline': 94,
  'lips': 182,
  'moustache': 4,
  'nose': 104},
 'Kristin Chenoweth': {'beard': 5,
  'cheek': 128,
  'ears': 13,
  'eye brows': 61,
  'eyes': 245,
  'hairline': 55,
  'lips': 233,
  'moustache': 7,
  'nose': 91},
 'Lorraine Bracco': {'beard': 8,
  'cheek': 141,
  'ears': 17,
  'eye brows': 62,
  'eyes': 184,
  'hairline': 68,
  'lips': 178,
  'moustache': 6,
  'nose': 174},
 'Matt Damon': {'beard': 13,
  'cheek': 71,
  'ears': 24,
  'eye brows': 71,
  'eyes': 223,
  'hairline': 68,
  'lips': 196,
  'moustache': 14,
  'nose': 158},
 'Michael Vartan': {'beard': 102,
  'cheek': 43,
  'ears': 31,
  'eye brows': 54,
  'eyes': 205,
  'hairline': 72,
  'lips': 88,
  'moustache': 49,
  'nose': 194},
 'Nicolas Cage': {'beard': 18,
  'cheek': 55,
  'ears': 32,
  'eye brows': 103,
  'eyes': 242,
  'hairline': 60,
  'lips': 137,
  'moustache': 20,
  'nose': 171},
 'Selena Gomez': {'beard': 3,
  'cheek': 172,
  'ears': 8,
  'eye brows': 46,
  'eyes': 223,
  'hairline': 51,
  'lips': 240,
  'moustache': 8,
  'nose': 87},
 'Steve Carell': {'beard': 32,
  'cheek': 57,
  'ears': 39,
  'eye brows': 135,
  'eyes': 181,
  'hairline': 37,
  'lips': 134,
  'moustache': 23,
  'nose': 200}}

In [79]:
true_saliencies['Alec Baldwin'].shape

(15, 227, 227)

In [80]:
actor_true_saliencies = {}

for actor in true_saliencies:
    actor_saliency = np.mean(true_saliencies[actor], axis=0)
    actor_true_saliencies[actor] = actor_saliency

In [81]:
male_actors = ['Daniel Radcliffe', 'Gerard Butler', 'Michael Vartan', 'Alec Baldwin', 'Steve Carell',\
                    'Bill Hader', 'Fran Drescher', 'Matt Damon', 'Nicolas Cage']
female_actors = ['Cheryl Hines', 'Selena Gomez', 'Angie Harmon', 'Lorraine Bracco', 'Kristin Chenoweth',\
                      'America Ferrera', 'Anne Hathaway', 'Jennifer Aniston']
gender ={}
for actor in male_actors:
    gender[actor] = 'male'
for actor in female_actors:
    gender[actor] = 'female' 

In [82]:
# creating turker count per actor
features = ['beard', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
actor_stats = {}
for actor in test_set:
    actor_stats[actor] = {}
    for feature in features:
        actor_stats[actor][feature] = 0  
    for stats in turker_count[actor]:
        for feature in features:
            try:
                 actor_stats[actor][feature] += stats[feature]
            except:
                continue

actor_stats_normalized = {}
for actor in actor_stats:
    actor_stats_normalized[actor] = {}
    total_count = sum(actor_stats[actor].values())
    for feature in features:
        actor_stats_normalized[actor][feature] = actor_stats[actor][feature]/total_count

ZeroDivisionError: division by zero

In [83]:
# creating turker coiunt per actor
features = ['beard', 'eye brows', 'eyes', 'hairline', 'lips', 'moustache', 'nose']
actor_stats = {}
for actor in test_set:
    actor_stats[actor] = {}
    for feature in features:
        actor_stats[actor][feature] = 0  
#     for stats in turker_count[actor]:
#         for feature in features:
#             try:
#                  actor_stats[actor][feature] += stats[feature]
#             except:
#                 continue
                
    stats = turker_count[actor]
    for feature in features:
        try:
             actor_stats[actor][feature] += stats[feature]
        except:
            continue

actor_stats_normalized = {}
for actor in actor_stats:
    actor_stats_normalized[actor] = {}
    total_count = sum(actor_stats[actor].values())
    for feature in features:
        actor_stats_normalized[actor][feature] = actor_stats[actor][feature]/total_count

In [84]:
def create_individual_prior(test_subject):

    if gender[test_subject] == 'male':
        actor_set = male_actors.copy()
    else:
        actor_set = female_actors.copy()
    
#     actor_set = male_actors.copy() + female_actors.copy()
    actor_set.remove(test_subject)
    prior = {}
    for feature in features:
        feature_stats = [actor_stats_normalized[actor][feature] for actor in actor_set]
        feature_prior = sum(feature_stats)/len(feature_stats)
        prior[feature] = feature_prior
    
    return prior

In [86]:
# avg_intensity_ratios = {}
# for feature in features:
#     feature_intensity_ratios = [predictions[actor][feature] for actor in sorted(test_set.keys())] 
#     avg_likelihood[feature] = sum(feature_likelihoods)/len(feature_likelihoods)
    
# for actor in predictions:
#     for feature in predictions[actor]:
#         predictions[actor][feature] /= avg_likelihood[feature]

In [87]:
def score(feature, rectangles, saliency_map):
    intensity_ratio_sum = 0
    for rec in rectangles[feature]:
        highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
        area = (rec[3]-rec[2])*(rec[1]-rec[0])
        intensity_ratio = (np.sum(highlight)/area)/np.sum(saliency_map)/(227**2)
        intensity_ratio_sum += intensity_ratio
    return intensity_ratio_sum/len(rectangles[feature])

In [88]:
scores = {}
deviations = {}
for actor in test_set.keys():
    scores[actor] = {}
    deviations[actor] = {}
    for feature in features:
        scores[actor][feature] = score(feature, mc_rectangle_samples, actor_true_saliencies[actor])

for feature in features:
    feature_scores = [scores[actor][feature] for actor in test_set.keys()]
    avg_feature_score = sum(feature_scores)/len(feature_scores)
    for actor in test_set.keys():
        deviations[actor][feature] = scores[actor][feature]/avg_feature_score

In [89]:
def likelihood(deviations):    
    probs = {}
    for actor in test_set.keys():
        probs[actor] = {}
        sum_deviations = np.sum(np.array(list(deviations[actor].values())))
        for feature in features:
            probs[actor][feature] = deviations[actor][feature]/sum_deviations
    return probs

In [90]:
predictions = likelihood(deviations)

In [91]:
def posterior(rectangles, saliency_map, test_subject):
    scores = {}
    prior = create_individual_prior(test_subject)
    for feature in features:
#         intensity_ratio_sum = 0
#         for rec in rectangles[feature]:
#             highlight = saliency_map[rec[2]:rec[3], rec[0]:rec[1]]
#             area = (rec[3]-rec[2])*(rec[1]-rec[0])
#             intensity_ratio = (np.sum(highlight)/area)/np.sum(saliency_map)/(227**2)
#             intensity_ratio_sum += intensity_ratio
#         scores[feature] = prior[feature]*intensity_ratio_sum/len(rectangles[feature])
        scores[feature] = prior[feature]*deviations[test_subject][feature]
    sum_scores = np.sum(np.array(list(scores.values())))
    probs = {}
    
    for feature in sorted(scores.keys()):
        probs[feature] = scores[feature]/sum_scores
    
    return probs

In [92]:
posterior_predictions = {}
for actor in sorted(test_set.keys()):
    saliency_map = actor_true_saliencies[actor]
    probs = posterior(mc_rectangle_samples, saliency_map, actor)
    posterior_predictions[actor]=probs

In [93]:
actor_stats_normalized

{'Alec Baldwin': {'beard': 0.014627659574468085,
  'eye brows': 0.125,
  'eyes': 0.3523936170212766,
  'hairline': 0.061170212765957445,
  'lips': 0.21808510638297873,
  'moustache': 0.014627659574468085,
  'nose': 0.21409574468085107},
 'America Ferrera': {'beard': 0.0014388489208633094,
  'eye brows': 0.1223021582733813,
  'eyes': 0.22158273381294963,
  'hairline': 0.0460431654676259,
  'lips': 0.37410071942446044,
  'moustache': 0.010071942446043165,
  'nose': 0.22446043165467625},
 'Angie Harmon': {'beard': 0.011347517730496455,
  'eye brows': 0.1773049645390071,
  'eyes': 0.29219858156028367,
  'hairline': 0.03829787234042553,
  'lips': 0.3049645390070922,
  'moustache': 0.01276595744680851,
  'nose': 0.16312056737588654},
 'Anne Hathaway': {'beard': 0.0050568900126422255,
  'eye brows': 0.11378002528445007,
  'eyes': 0.32364096080910243,
  'hairline': 0.05309734513274336,
  'lips': 0.3944374209860936,
  'moustache': 0.0025284450063211127,
  'nose': 0.10745891276864729},
 'Bill Ha

In [94]:
# main problems are with turker data
# maybe imbalanced, also tend to label similar feature
# maybe should try new set of actors who have very distinct features to do anchoring
# people choocse what seems to get highlighted in that image, but not what is salient about the person
# probaly better to frame questionas as what is salient about this individual

In [95]:
import operator

actors = sorted(test_set.keys())

f, axes = plt.subplots(17, 4, figsize=(60,170))

for i in range(len(actors)):
    actor = actors[i]
    ax = axes[i]

    ax[0].set_title(actor)
    ax[0].axis('off')
    ax[0].imshow(actor_true_saliencies[actor])
        
    prediction = posterior_predictions[actor]
    turker_data = actor_stats_normalized[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    sorted_turker_data.reverse()
    
    X = np.arange(len(turker_data))
    ax[1].set_ylim([0, 0.7])
    ax[1].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Posterior Prediction')
    ax[1].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[1].legend()
    ax[1].set_xticks(X)
    ax[1].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[1].set_title('turkers vs posterior', fontsize=17)
    
    
    prediction = create_individual_prior(actor)
    X = np.arange(len(turker_data))
    ax[2].set_ylim([0, 0.7])
    ax[2].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Prior Prediction')
    ax[2].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[2].legend()
    ax[2].set_xticks(X)
    ax[2].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[2].set_title('turkers vs prior', fontsize=17)
    
    prediction = predictions[actor]
    X = np.arange(len(turker_data))
    ax[3].set_ylim([0, 0.7])
    ax[3].bar(X, [prediction[key[0]] for key in sorted_turker_data],\
           width=0.2, color='b', align='center', label='Backprop Prediction')
    ax[3].bar(X-0.2, [turker_data[key[0]] for key in sorted_turker_data],\
           width=0.2, color='g', align='center', label='Turkers')
    ax[3].legend()
    ax[3].set_xticks(X)
    ax[3].set_xticklabels([key[0] for key in sorted_turker_data])
    ax[3].set_title('turkers vs backprop', fontsize=17)

    print(actor)
    
    
plt.savefig('robust_predictions_size_feature_normalized.png')
plt.close()

Alec Baldwin
America Ferrera
Angie Harmon
Anne Hathaway
Bill Hader
Cheryl Hines
Daniel Radcliffe
Fran Drescher
Gerard Butler
Jennifer Aniston
Kristin Chenoweth
Lorraine Bracco
Matt Damon
Michael Vartan
Nicolas Cage
Selena Gomez
Steve Carell


In [96]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = posterior_predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.99103120896511487, pvalue=1.4561252916129422e-05)
America Ferrera
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Angie Harmon
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Anne Hathaway
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Bill Hader
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Cheryl Hines
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Daniel Radcliffe
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Fran Drescher
SpearmanrResult(correlation=0.78571428571428592, pvalue=0.036238462679827131)
Gerard Butler
SpearmanrResult(correlation=0.39285714285714296, pvalue=0.38331687042697266)
Jennifer Aniston
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Kristin Chenoweth
SpearmanrResult(correlation=0.92857142857142883, pvalue

In [97]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = create_individual_prior(actor)
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.99103120896511487, pvalue=1.4561252916129422e-05)
America Ferrera
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Angie Harmon
SpearmanrResult(correlation=0.96428571428571452, pvalue=0.00045414916919416892)
Anne Hathaway
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Bill Hader
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Cheryl Hines
SpearmanrResult(correlation=0.92857142857142883, pvalue=0.0025194724037946874)
Daniel Radcliffe
SpearmanrResult(correlation=0.85714285714285732, pvalue=0.01369732661532562)
Fran Drescher
SpearmanrResult(correlation=0.78571428571428592, pvalue=0.036238462679827131)
Gerard Butler
SpearmanrResult(correlation=0.42857142857142866, pvalue=0.33736831108582399)
Jennifer Aniston
SpearmanrResult(correlation=0.8928571428571429, pvalue=0.0068071874089353918)
Kristin Chenoweth
SpearmanrResult(correlation=0.96428571428571452, pvalue=

In [98]:
from scipy.stats import spearmanr
cors = []
pvals = []
for actor in sorted(test_set.keys()):
    print(actor)
    turker_data = actor_stats[actor]
    sorted_turker_data = sorted(turker_data.items(), key=operator.itemgetter(1))
    prediction = predictions[actor]
    predicted = np.array([prediction[key[0]] for key in sorted_turker_data])
    turker = np.array([turker_data[key[0]] for key in sorted_turker_data])
    cor, pval = spearmanr(predicted, turker) 
    cors.append(cor)
    pvals.append(pval)
    print(spearmanr(predicted, turker))
print('correlation')
print(np.mean(np.array(cors)))    
print('pvalue')
print(np.mean(np.array(pvals)))

Alec Baldwin
SpearmanrResult(correlation=0.10811249552346709, pvalue=0.81753326562411821)
America Ferrera
SpearmanrResult(correlation=0.5, pvalue=0.25316999510032273)
Angie Harmon
SpearmanrResult(correlation=0.42857142857142866, pvalue=0.33736831108582399)
Anne Hathaway
SpearmanrResult(correlation=-0.14285714285714288, pvalue=0.75994530021809292)
Bill Hader
SpearmanrResult(correlation=0.071428571428571438, pvalue=0.87904819314815408)
Cheryl Hines
SpearmanrResult(correlation=0.10714285714285716, pvalue=0.81915085556999123)
Daniel Radcliffe
SpearmanrResult(correlation=0.75000000000000022, pvalue=0.052181400457057762)
Fran Drescher
SpearmanrResult(correlation=-0.14285714285714288, pvalue=0.75994530021809292)
Gerard Butler
SpearmanrResult(correlation=0.14285714285714288, pvalue=0.75994530021809292)
Jennifer Aniston
SpearmanrResult(correlation=0.21428571428571433, pvalue=0.64451158102072026)
Kristin Chenoweth
SpearmanrResult(correlation=-0.5, pvalue=0.25316999510032273)
Lorraine Bracco
Spea